In [9]:
# import dependencies
import pandas as pd
from pathlib import Path
import sqlite3
!pip install pandas psycopg2
!pip install plotly
import plotly.express as px
import psycopg2
from keys import post_username, post_password

# import CSV
climate_impact_data = Path("Resources/climate_change_impact_on_agriculture_2024.csv")

In [10]:
# Create a DataFrame
climate_impact_df = pd.read_csv(climate_impact_data)
climate_impact_df.head()

,Year,Country,Region,Crop_Type,Average_Temperature_C,Total_Precipitation_mm,CO2_Emissions_MT,Crop_Yield_MT_per_HA,Extreme_Weather_Events,Irrigation_Access_%,Pesticide_Use_KG_per_HA,Fertilizer_Use_KG_per_HA,Soil_Health_Index,Adaptation_Strategies,Economic_Impact_Million_USD
0,2001,India,West Bengal,Corn,1.55,447.06,15.22,1.737,8,14.54,10.08,14.78,83.25,Water Management,808.13
1,2024,China,North,Corn,3.23,2913.57,29.82,1.737,8,11.05,33.06,23.25,54.02,Crop Rotation,616.22
2,2001,France,Ile-de-France,Wheat,21.11,1301.74,25.75,1.719,5,84.42,27.41,65.53,67.78,Water Management,796.96
3,2001,Canada,Prairies,Coffee,27.85,1154.36,13.91,3.890,5,94.06,14.38,87.58,91.39,No Adaptation,790.32
4,1998,India,Tamil Nadu,Sugarcane,2.19,1627.48,11.81,1.080,9,95.75,44.35,88.08,49.61,Crop Rotation,401.72


In [11]:
climate_impact_df['Region'].unique()

array(['West Bengal', 'North', 'Ile-de-France', 'Prairies', 'Tamil Nadu',
       'Midwest', 'Northeast', 'New South Wales', 'Punjab', 'North West',
       'South East', 'Grand Est', 'Northwestern', 'Siberian', 'Northwest',
       'Victoria', 'Nouvelle-Aquitaine', 'South', 'Quebec', 'Southeast',
       'Ontario', 'East', 'Pampas', 'Western Australia', 'Volga',
       'Maharashtra', 'Provence-Alpes-Cote d’Azur', 'West', 'Central',
       'North Central', 'Patagonia', 'Queensland', 'South West',
       'British Columbia'], dtype=object)

In [12]:
len(climate_impact_df['Region'].unique())

34

In [13]:
climate_impact_df['Country'].unique()

array(['India', 'China', 'France', 'Canada', 'USA', 'Argentina',
       'Australia', 'Nigeria', 'Russia', 'Brazil'], dtype=object)

In [14]:
len(climate_impact_df['Country'].unique())

10

In [15]:
group_df = climate_impact_df.groupby('Country')['Region'].nunique().reset_index()

In [16]:
group_df

,Country,Region
0,Argentina,4
1,Australia,4
2,Brazil,4
3,Canada,4
4,China,4
5,France,4
6,India,4
7,Nigeria,4
8,Russia,4
9,USA,4


In [17]:
grouped_df = climate_impact_df.groupby("Country")["Region"].unique().reset_index()

pd.set_option('display.max_colwidth', None)
grouped_df

,Country,Region
0,Argentina,"[Northeast, Northwest, Pampas, Patagonia]"
1,Australia,"[New South Wales, Victoria, Western Australia, Queensland]"
2,Brazil,"[North, Northeast, Southeast, South]"
3,Canada,"[Prairies, Quebec, Ontario, British Columbia]"
4,China,"[North, East, South, Central]"
5,France,"[Ile-de-France, Grand Est, Nouvelle-Aquitaine, Provence-Alpes-Cote d’Azur]"
6,India,"[West Bengal, Tamil Nadu, Punjab, Maharashtra]"
7,Nigeria,"[North West, South East, North Central, South West]"
8,Russia,"[Northwestern, Siberian, Volga, Central]"
9,USA,"[Midwest, Northeast, South, West]"


In [18]:
# Check for unique crops per country, region, and year
unique_crops = climate_impact_df.groupby(['Country', 'Region', 'Year'])['Crop_Type'].nunique()

# Display the results
unique_crops

Country    Region     Year
Argentina  Northeast  1990    8
                      1991    8
                      1992    6
                      1993    3
                      1994    6
                             ..
USA        West       2020    4
                      2021    8
                      2022    1
                      2023    7
                      2024    6
Name: Crop_Type, Length: 1398, dtype: int64

Investigate the Crop_Type column, find the number of unique entries.

In [19]:
total_unique_crops = climate_impact_df['Crop_Type'].unique()
total_unique_crops

array(['Corn', 'Wheat', 'Coffee', 'Sugarcane', 'Fruits', 'Rice', 'Barley',
       'Vegetables', 'Soybeans', 'Cotton'], dtype=object)

In [20]:
len(total_unique_crops)

10

In [21]:
# Creating the pivot table
pivot_table = climate_impact_df.pivot_table(
    index=['Country', 'Region'],   # Rows
    columns=['Year', 'Crop_Type'],  # Columns
    aggfunc='count',                # Change this if you want a different aggregation
    fill_value=0                    # Replace NaN with 0
)

pivot_table.head()

Adaptation_Strategies                            \
Year                                       1990                             
Crop_Type                                Barley Coffee Corn Cotton Fruits   
Country   Region                                                            
Argentina Northeast                           1      1    3      1      2   
          Northwest                           0      1    0      0      1   
          Pampas                              1      1    0      0      1   
          Patagonia                           1      0    0      1      0   
Australia New South Wales                     0      0    1      2      1   

                                                                    ...  \
Year                                                                ...   
Crop_Type                 Rice Soybeans Sugarcane Vegetables Wheat  ...   
Country   Region                                                    ...   
Argentina Northeast          1        0         1          1     0  ...   
          Northwest          1        0         0          0     0  ...   
          Pampas             1        1         0          1     2  ...   
          Patagonia          0        1         0          0     0  ...   
Australia New South Wales    0        0         1          0     1  ...   

                          Total_Precipitation_mm                            \
Year                                        2024                             
Crop_Type                                 Barley Coffee Corn Cotton Fruits   
Country   Region                                                             
Argentina Northeast                            0      1    2      0      3   
          Northwest                            1      0    1      1      2   
          Pampas                               0      0    0      2      1   
          Patagonia                            0      2    0      1      0   
Australia New South Wales                      0      1    2      2      1   

                                                                    
Year                                                                
Crop_Type                 Rice Soybeans Sugarcane Vegetables Wheat  
Country   Region                                                    
Argentina Northeast          0        0         3          0     0  
          Northwest          0        0         0          1     1  
          Pampas             1        0         0          0     0  
          Patagonia          2        1         0          0     0  
Australia New South Wales    0        3         3          0     0  

[5 rows x 3850 columns]

Even in the `.head.()` preview, you can see that there are multiples some occurences of crop type, per year, per region, per country. This was done to test if a composite key of year, country, region, and crop type could be made for importing to an SQL database. A bit of the horse before the cart, because I think any actions we take doing ETL should be done after loading this CSV dataset from an SQL database.

Have been using Excel pivot tables to view the data, check for issues. Mostly trying to see if I can find a primary key, but I don't think there is one. Each row is basically it's own primary key.

Now how do I use sqlite to find this database? I need to import it here.

In [22]:
# use psycopg2 to connect to local library
conn = psycopg2.connect(
    dbname='team6-project3_db',
    user= post_username,
    password= post_password,
    host='localhost',
    port='5432'
)

In [23]:
query = "SELECT * FROM climate_impact_agriculture;"

In [24]:
new_climate_df = pd.read_sql_query(query, conn)

C:\Users\nbart.DESKTOP-3OF7M8N\AppData\Local\Temp\ipykernel_24300\3604977624.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  new_climate_df = pd.read_sql_query(query, conn)


In [25]:
conn.close()

In [26]:
new_climate_df.head()

,year,country,region,crop_type,average_temperature_c,total_precipitation_mm,co2_emissions_mt,crop_yield_mt_per_ha,extreme_weather_events,Irrigation_Access_%,pesticide_use_kg_per_ha,fertilizer_use_kg_per_ha,soil_health_index,adaptation_strategies,economic_impact_million_usd
0,2001,India,West Bengal,Corn,1.55,447.06,15.22,1.737,8,14.54,10.08,14.78,83.25,Water Management,808.13
1,2024,China,North,Corn,3.23,2913.57,29.82,1.737,8,11.05,33.06,23.25,54.02,Crop Rotation,616.22
2,2001,France,Ile-de-France,Wheat,21.11,1301.74,25.75,1.719,5,84.42,27.41,65.53,67.78,Water Management,796.96
3,2001,Canada,Prairies,Coffee,27.85,1154.36,13.91,3.890,5,94.06,14.38,87.58,91.39,No Adaptation,790.32
4,1998,India,Tamil Nadu,Sugarcane,2.19,1627.48,11.81,1.080,9,95.75,44.35,88.08,49.61,Crop Rotation,401.72


Created a DataFrame with DB data. Others won't be able to replicate this because the DB is local, they can't run the above code. One solution is a cloud server but that seems like a lot of work. Instead try creating an sqlite file to save on the repo.

In [27]:
# Create or load DB file - connect to DB
sqlite_conn = sqlite3.connect('climate_impact_agriculture.db')

# Load table into DB format, using SQLITE connection
new_climate_df.to_sql('climate_impact_agriculture', sqlite_conn, if_exists='replace', index=False)

# Close connection
sqlite_conn.close()

OK we created the DB file in a convoluted way that was surely unnecessary... but it's there and can help us create a DF, which we've already done twice...
I will comment out the part where I import it from my computer? I think it's important they see where it came from, but no one else will be able to run it. The alternative is to create the cloud server... I've already done it this way but, it's a strange method that doesn't really make sense. 

In [28]:
# Connect to DB file
sqlite_conn = sqlite3.connect('climate_impact_agriculture.db')

# Use pandas to read into an DataFrame
third_climate_df = pd.read_sql_query("SELECT * FROM climate_impact_agriculture;", sqlite_conn)

# Close connection
sqlite_conn.close()

# View df
third_climate_df.head()

,year,country,region,crop_type,average_temperature_c,total_precipitation_mm,co2_emissions_mt,crop_yield_mt_per_ha,extreme_weather_events,Irrigation_Access_%,pesticide_use_kg_per_ha,fertilizer_use_kg_per_ha,soil_health_index,adaptation_strategies,economic_impact_million_usd
0,2001,India,West Bengal,Corn,1.55,447.06,15.22,1.737,8,14.54,10.08,14.78,83.25,Water Management,808.13
1,2024,China,North,Corn,3.23,2913.57,29.82,1.737,8,11.05,33.06,23.25,54.02,Crop Rotation,616.22
2,2001,France,Ile-de-France,Wheat,21.11,1301.74,25.75,1.719,5,84.42,27.41,65.53,67.78,Water Management,796.96
3,2001,Canada,Prairies,Coffee,27.85,1154.36,13.91,3.890,5,94.06,14.38,87.58,91.39,No Adaptation,790.32
4,1998,India,Tamil Nadu,Sugarcane,2.19,1627.48,11.81,1.080,9,95.75,44.35,88.08,49.61,Crop Rotation,401.72


In [29]:
# Create a choropleth plot with time series
fig = px.choropleth(
    third_climate_df,
    locations='country', 
    color='co2_emissions_mt',  # Column with the data you want to plot (e.g., CO2 emissions)
    hover_name='region',  # Show regions on hover
    animation_frame='year',  # Column for time series (e.g., 'Year')
    color_continuous_scale='Viridis',  # Color scale for emissions
    projection='natural earth',  # Type of projection (world map)
    title='CO2 Emissions Over Time by Region'
)

# Update layout for better appearance
fig.update_layout(
    coloraxis_colorbar=dict(
        title="CO2 Emissions (Mt)",
    )
)

# Show the plot
fig.show()

At least it's showing something? Time is random, needs to be sorted. Other countries need to be listed. This could be a cool looking visual for the project.

In [30]:
# Set up database connection parameters
dbname = "team6_project3_db"
user = "postgres"
password = "postgres"
host = "database-1.croamw4iqxpi.us-east-2.rds.amazonaws.com"  # or the IP address of your database server
port = "5432" 

In [31]:
import pandas as pd
from sqlalchemy import create_engine
# Define the database connection parameters
db_user = "postgres"  # replace with your PostgreSQL username
db_password = "postgres"  # replace with your PostgreSQL password
db_host = "database-1.croamw4iqxpi.us-east-2.rds.amazonaws.com"  # replace with your host
db_port = '5432'  # replace with your port
db_name = "team6_project3_db"  # replace with your PostgreSQL database name
# Create the database engine
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
# Example DataFrame to load
data = {'column1': [1, 2, 3], 'column2': ['A', 'B', 'C']}
df = pd.DataFrame(data)
# Load DataFrame to PostgreSQL
table_name = 'dummy_table2'  # Replace with your table name
third_climate_df.to_sql(table_name, engine, if_exists='replace', index=False)  # Set if_exists to 'replace', 'append', or 'fail'
print("Data loaded successfully!")

Data loaded successfully!


In [7]:
# Query and load into DataFrame
with engine.connect() as connection:
    data = pd.read_sql("SELECT * FROM dummy_table", con=connection)

data

,column1,column2
0,1,A
1,2,B
2,3,C
